## <b>■ 딥러닝 복습</b>
    1장. numpy
    2장. 퍼셉트론
    3장. 3층 신경망 구현
    4장. 2층 신경망 구현(수치미분)
    5장. 2층 신경망 구현(오차역전파)      tensorflow 1.x -> tensorflow 2.x
    --------------------------------------------------------------------
    6장. 신경망 학습시키는 기술들
    7장. CNN을 이용한 신경망 구현
    -------------------------------------------------------------------- 자전거 타는 법

    6장. 신경망 학습시키는 기술들
        1. 언더피팅 방지하는 방법
            - 가중치 초기값 선정
                ① Xavier
                ② He
            - 배치 정규화
        2. 오버피팅 방지하는 방법
### <b>■ 텐서플로우로 가중치 초기값 선정하는 방법</b>
    1. Xavier
![fig6-14](fig6-13.png)

$$ {{1} \over {\sqrt{n}}} \cdot \rm{np.random.randn(r,c)}$$

In [ ]:
W1 = tf.get_variable(name='W1', shape=[784,50], initializer = tf.contrib.layers.xavier_initializer())

    2. He 가중치 초기값 구성
$$ \sqrt{{{2} \over {n}}} \cdot \rm{np.random.randn(r,c)} $$

In [ ]:
W1 = tf.get_variable(name="W1", shape=[784,50], initializer = tf.contrib.layers.variance_scaling_initializer())

#### 예제1. 어제 마지막 문제로 만들었던 텐서 플로우로 구현한 신경망 코드에 가중치 초기값을 xavier로 해서 구현하시오

In [9]:
import tensorflow as tf
import numpy as np
import warnings
import os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

tf.reset_default_graph()

# 은닉1층
x = tf.placeholder('float',[None,784])
W1 = tf.get_variable(name='W1', shape=[784,50], initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.ones([1,50]))

y = tf.matmul(x, W1) + b1
y_hat = tf.nn.relu(y)

# 출력층
W2 = tf.get_variable(name='W2', shape=[50,10], initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.ones([1,10]))

z = tf.matmul(y_hat,W2) + b2
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder('float',[None,10]) # 정답 데이터를 담을 배열
y_label = tf.argmax(y_onehot, axis=1)

correction_prediction = tf.equal(y_predict, y_label)
accuracy = tf.reduce_mean(tf.cast(correction_prediction,'float'))

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.0000001), axis=1)
rs = tf.reduce_mean(loss)

# 학습
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# 변수 초기화
init = tf.global_variables_initializer()

# 그래프 실행
sess = tf.Session()
sess.run(init)

for i in range(1,601*20):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    batch_x_test, batch_y_test = mnist.test.next_batch(100)
    sess.run(train, feed_dict={x:batch_xs, y_onehot:batch_ys})
    if not i % 600:
        print(i//600,"에폭 훈련데이터 정확도 : ",sess.run(accuracy, feed_dict={x:batch_xs, y_onehot:batch_ys}),"\t","테스트 데이터 정확도:", sess.run(accuracy, feed_dict={x:batch_x_test, y_onehot:batch_y_test}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
{1} 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.92
{2} 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.98
{3} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.99
{4} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.94
{5} 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.97
{6} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
{7} 에폭 훈련데이터 정확도 :  0.96 	 테스트 데

### ※ 문제137. 이번에는 가중치 초기값을 He로 해서 수행하시오

In [10]:
import tensorflow as tf
import numpy as np
import warnings
import os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

tf.reset_default_graph() # 텐서 그래프 초기화 하는 코드

# 은닉1층
x = tf.placeholder('float',[None,784])
W1 = tf.get_variable(name="W1", shape=[784,50], initializer = tf.contrib.layers.variance_scaling_initializer())
b1 = tf.Variable(tf.ones([1,50]))

y = tf.matmul(x, W1) + b1
y_hat = tf.nn.relu(y)

# 출력층
W2 = tf.get_variable(name="W2", shape=[50,10], initializer = tf.contrib.layers.variance_scaling_initializer())
b2 = tf.Variable(tf.ones([1,10]))

z = tf.matmul(y_hat,W2) + b2
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder('float',[None,10]) # 정답 데이터를 담을 배열
y_label = tf.argmax(y_onehot, axis=1)

correction_prediction = tf.equal(y_predict, y_label)
accuracy = tf.reduce_mean(tf.cast(correction_prediction,'float'))

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.0000001), axis=1)
rs = tf.reduce_mean(loss)

# 학습
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# 변수 초기화
init = tf.global_variables_initializer()

# 그래프 실행
sess = tf.Session()
sess.run(init)

for i in range(1,601*20):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    batch_x_test, batch_y_test = mnist.test.next_batch(100)
    sess.run(train, feed_dict={x:batch_xs, y_onehot:batch_ys})
    if not i % 600:
        print(i//600,"에폭 훈련데이터 정확도 : ",sess.run(accuracy, feed_dict={x:batch_xs, y_onehot:batch_ys}),"\t","테스트 데이터 정확도:", sess.run(accuracy, feed_dict={x:batch_x_test, y_onehot:batch_y_test}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
{1} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.96
{2} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 1.0
{3} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
{4} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.99
{5} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
{6} 에폭 훈련데이터 정확도 :  0.97 	 테스트 데이터 정확도: 0.93
{7} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.98
{8} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.97
{9} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.97
{10} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.98
{11} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
{12} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
{13} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.95
{14} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
{15} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
{16} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.96
{17} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
{18} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.

### ※ 문제138. 위의 2층 신경망을 3층 신경망으로 변경하시오
    기존층 : 입력층 -------> 은닉1층 ------> 출력층
             784             100             10
    변경후 : 입력층 -------> 은닉1층 ------> 은닉2층 -------> 출력층
             784             100             50              10

In [12]:
import tensorflow as tf
import numpy as np
import warnings
import os
from tensorflow.examples.tutorials.mnist import input_data
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

tf.reset_default_graph() # 텐서 그래프 초기화 하는 코드

# 은닉1층
x = tf.placeholder('float',[None,784])
W1 = tf.get_variable(name="W1", shape=[784,100], initializer = tf.contrib.layers.variance_scaling_initializer())
b1 = tf.Variable(tf.ones([1,100]))

y = tf.matmul(x, W1) + b1
y_hat = tf.nn.relu(y)

# 은닉2층
W2 = tf.get_variable(name="W2", shape=[100,50], initializer = tf.contrib.layers.variance_scaling_initializer())
b2 = tf.Variable(tf.ones([1,50]))

y2 = tf.matmul(y_hat,W2) + b2
y2_hat = tf.nn.relu(y2)

# 출력층
W3 = tf.get_variable(name="W3", shape=[50,10], initializer = tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.ones([1,10]))

z = tf.matmul(y2_hat,W3) + b3
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder('float',[None,10]) # 정답 데이터를 담을 배열
y_label = tf.argmax(y_onehot, axis=1)

correction_prediction = tf.equal(y_predict, y_label)
accuracy = tf.reduce_mean(tf.cast(correction_prediction,'float'))

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.0000001), axis=1)
rs = tf.reduce_mean(loss)

# 학습
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# 변수 초기화
init = tf.global_variables_initializer()

# 그래프 실행
sess = tf.Session()
sess.run(init)

for i in range(1,601*20):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    batch_x_test, batch_y_test = mnist.test.next_batch(100)
    sess.run(train, feed_dict={x:batch_xs, y_onehot:batch_ys})
    if not i % 600:
        print(i//600,"에폭 훈련데이터 정확도 : ",sess.run(accuracy, feed_dict={x:batch_xs, y_onehot:batch_ys}),"\t","테스트 데이터 정확도:", sess.run(accuracy, feed_dict={x:batch_x_test, y_onehot:batch_y_test}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
{1} 에폭 훈련데이터 정확도 :  0.91 	 테스트 데이터 정확도: 0.95
{2} 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.97
{3} 에폭 훈련데이터 정확도 :  0.97 	 테스트 데이터 정확도: 0.98
{4} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.93
{5} 에폭 훈련데이터 정확도 :  0.97 	 테스트 데이터 정확도: 0.97
{6} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.96
{7} 에폭 훈련데이터 정확도 :  0.97 	 테스트 데이터 정확도: 0.96
{8} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.96
{9} 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.96
{10} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.97
{11} 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.97
{12} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
{13} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
{14} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
{15} 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.94
{16} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.93
{17} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.98
{18} 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확

### <b>■ 텐서 플로우로 배치 정규화 구현하는 방법</b>
    배치 정규화 - 신경망 학습시 가중치 값의 데이터가 골고루 분산될 수 있도록 강제화 하는 장치
        층이 깊어져도 가중치의 정규분포를 계속 유지할 수 있도록 층마다 강제화 하는 장치
    
```python
batch_z1 = tf.contrib.layers.batch_norm(z1, True)
```
    Affine1 ------> 배치 정규화 ------> ReLU
     (z1)
     
#### 예제1. 지금까지 완성한 신경망에 배치 정규화를 은닉 1층에 구현하시오

In [13]:
import tensorflow as tf
import numpy as np
import warnings
import os
from tensorflow.examples.tutorials.mnist import input_data
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

tf.reset_default_graph() # 텐서 그래프 초기화 하는 코드

# 은닉1층
x = tf.placeholder('float',[None,784])
W1 = tf.get_variable(name="W1", shape=[784,100], initializer = tf.contrib.layers.variance_scaling_initializer())
b1 = tf.Variable(tf.ones([1,100]))

y1 = tf.matmul(x, W1) + b1

batch_y1 = tf.contrib.layers.batch_norm(y1, True)

y1_hat = tf.nn.relu(batch_y1)

# 은닉2층
W2 = tf.get_variable(name="W2", shape=[100,50], initializer = tf.contrib.layers.variance_scaling_initializer())
b2 = tf.Variable(tf.ones([1,50]))

y2 = tf.matmul(y1_hat,W2) + b2
y2_hat = tf.nn.relu(y2)

# 출력층
W3 = tf.get_variable(name="W3", shape=[50,10], initializer = tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.ones([1,10]))

z = tf.matmul(y2_hat,W3) + b3
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder('float',[None,10]) # 정답 데이터를 담을 배열
y_label = tf.argmax(y_onehot, axis=1)

correction_prediction = tf.equal(y_predict, y_label)
accuracy = tf.reduce_mean(tf.cast(correction_prediction,'float'))

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.0000001), axis=1)
rs = tf.reduce_mean(loss)

# 학습
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# 변수 초기화
init = tf.global_variables_initializer()

# 그래프 실행
sess = tf.Session()
sess.run(init)

for i in range(1,601*20):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    batch_x_test, batch_y_test = mnist.test.next_batch(100)
    sess.run(train, feed_dict={x:batch_xs, y_onehot:batch_ys})
    if not i % 600:
        print(i//600,"에폭 훈련데이터 정확도 : ",sess.run(accuracy, feed_dict={x:batch_xs, y_onehot:batch_ys}),"\t","테스트 데이터 정확도:", sess.run(accuracy, feed_dict={x:batch_x_test, y_onehot:batch_y_test}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.97
2 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
3 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.98
4 에폭 훈련데이터 정확도 :  0.96 	 테스트 데이터 정확도: 0.98
5 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
6 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
7 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.94
8 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.94
9 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
10 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
11 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.98
12 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
13 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.93
14 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
15 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.95
16 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 1.0
17 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
18 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
19 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도

### ※ 문제139. 은닉 2층에도 배치 정규화를 적용하시오

In [14]:
import tensorflow as tf
import numpy as np
import warnings
import os
from tensorflow.examples.tutorials.mnist import input_data
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

tf.reset_default_graph() # 텐서 그래프 초기화 하는 코드

# 은닉1층
x = tf.placeholder('float',[None,784])
W1 = tf.get_variable(name="W1", shape=[784,100], initializer = tf.contrib.layers.variance_scaling_initializer())
b1 = tf.Variable(tf.ones([1,100]))

y1 = tf.matmul(x, W1) + b1

batch_y1 = tf.contrib.layers.batch_norm(y1, True)

y1_hat = tf.nn.relu(batch_y1)

# 은닉2층
W2 = tf.get_variable(name="W2", shape=[100,50], initializer = tf.contrib.layers.variance_scaling_initializer())
b2 = tf.Variable(tf.ones([1,50]))

y2 = tf.matmul(y1_hat,W2) + b2

batch_y2 = tf.contrib.layers.batch_norm(y2, True)

y2_hat = tf.nn.relu(batch_y2)

# 출력층
W3 = tf.get_variable(name="W3", shape=[50,10], initializer = tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.ones([1,10]))

z = tf.matmul(y2_hat,W3) + b3
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder('float',[None,10]) # 정답 데이터를 담을 배열
y_label = tf.argmax(y_onehot, axis=1)

correction_prediction = tf.equal(y_predict, y_label)
accuracy = tf.reduce_mean(tf.cast(correction_prediction,'float'))

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.0000001), axis=1)
rs = tf.reduce_mean(loss)

# 학습
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# 변수 초기화
init = tf.global_variables_initializer()

# 그래프 실행
sess = tf.Session()
sess.run(init)

for i in range(1,601*20):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    batch_x_test, batch_y_test = mnist.test.next_batch(100)
    sess.run(train, feed_dict={x:batch_xs, y_onehot:batch_ys})
    if not i % 600:
        print(i//600,"에폭 훈련데이터 정확도 : ",sess.run(accuracy, feed_dict={x:batch_xs, y_onehot:batch_ys}),"\t","테스트 데이터 정확도:", sess.run(accuracy, feed_dict={x:batch_x_test, y_onehot:batch_y_test}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.99
2 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.96
3 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.95
4 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
5 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
6 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
7 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.97
8 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
9 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
10 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.97
11 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 1.0
12 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
13 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
14 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
15 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 1.0
16 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
17 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
18 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
19 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 

In [14]:
import tensorflow as tf
import numpy as np
import warnings
import os
from tensorflow.examples.tutorials.mnist import input_data
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

tf.reset_default_graph() # 텐서 그래프 초기화 하는 코드

# 은닉1층
x = tf.placeholder('float',[None,784])
x1 = tf.reshape(x,[-1,28,28,1]) # 흑백사진, 1층, batch 개수를 모르므로 -1. 2차원 -> 4차원으로 변경

# Convolution 1층
W1 = tf.Variable(tf.random_normal)

W1 = tf.get_variable(name="W1", shape=[784,100], initializer = tf.contrib.layers.variance_scaling_initializer())
b1 = tf.Variable(tf.ones([1,100]))

y1 = tf.matmul(x, W1) + b1

batch_y1 = tf.contrib.layers.batch_norm(y1, True)

y1_hat = tf.nn.relu(batch_y1)

# 은닉2층
W2 = tf.get_variable(name="W2", shape=[100,50], initializer = tf.contrib.layers.variance_scaling_initializer())
b2 = tf.Variable(tf.ones([1,50]))

y2 = tf.matmul(y1_hat,W2) + b2

batch_y2 = tf.contrib.layers.batch_norm(y2, True)

y2_hat = tf.nn.relu(batch_y2)

# 출력층
W3 = tf.get_variable(name="W3", shape=[50,10], initializer = tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.ones([1,10]))

z = tf.matmul(y2_hat,W3) + b3
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder('float',[None,10]) # 정답 데이터를 담을 배열
y_label = tf.argmax(y_onehot, axis=1)

correction_prediction = tf.equal(y_predict, y_label)
accuracy = tf.reduce_mean(tf.cast(correction_prediction,'float'))

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.0000001), axis=1)
rs = tf.reduce_mean(loss)

# 학습
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# 변수 초기화
init = tf.global_variables_initializer()

# 그래프 실행
sess = tf.Session()
sess.run(init)

for i in range(1,601*20):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    batch_x_test, batch_y_test = mnist.test.next_batch(100)
    sess.run(train, feed_dict={x:batch_xs, y_onehot:batch_ys})
    if not i % 600:
        print(i//600,"에폭 훈련데이터 정확도 : ",sess.run(accuracy, feed_dict={x:batch_xs, y_onehot:batch_ys}),"\t","테스트 데이터 정확도:", sess.run(accuracy, feed_dict={x:batch_x_test, y_onehot:batch_y_test}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.99
2 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 0.96
3 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.95
4 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
5 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
6 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
7 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.97
8 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
9 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.99
10 에폭 훈련데이터 정확도 :  0.98 	 테스트 데이터 정확도: 0.97
11 에폭 훈련데이터 정확도 :  0.99 	 테스트 데이터 정확도: 1.0
12 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
13 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
14 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
15 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 1.0
16 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.98
17 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
18 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 0.97
19 에폭 훈련데이터 정확도 :  1.0 	 테스트 데이터 정확도: 